# Get all content from Netflix in Sweden -- DEPRECATED

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import time

# Set up the Selenium WebDriver (use appropriate WebDriver for your browser)
driver = webdriver.Chrome()

# Open the page
driver.get('https://www.justwatch.com/se/leverantör/netflix')

# Simulate scrolling
SCROLL_PAUSE_TIME = 2  # seconds

# Get the initial height of the page
last_height = driver.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to the bottom
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    
    # Wait for new content to load
    time.sleep(SCROLL_PAUSE_TIME)
    
    # Calculate new scroll height and compare with the last scroll height
    new_height = driver.execute_script("return document.body.scrollHeight")
    
    # If the height hasn't changed, we've reached the bottom of the page
    if new_height == last_height:
        break
    last_height = new_height

# Once all content is loaded, parse the page with BeautifulSoup
soup = BeautifulSoup(driver.page_source, 'html.parser')

# Find the title grid and extract the titles
title_grid = soup.find('div', class_='title-list-grid')

if title_grid:
    title_elements = title_grid.find_all('div', {'data-title': True})
    
    titles = [element['data-title'] for element in title_elements]
    
    for title in titles:
        print(title)
else:
    print("No titles found.")

# Close the driver
driver.quit()


# First Try

In [2]:
import pandas as pd

imdb = pd.read_csv('data/title.basics.tsv', sep='\t')
imdb = imdb[(imdb['titleType'] == 'movie') | (imdb['titleType'] == 'tvSeries')]
imdb['primaryTitle'] = imdb['primaryTitle'].str.lower()

/tmp/ipykernel_166093/2601532259.py:3: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  imdb = pd.read_csv('data/title.basics.tsv', sep='\t')


# Scrap IMDB

In [3]:
netflix = pd.read_csv('data/netflix_titles.csv')
netflix['type'] = netflix['type'].apply(lambda x: 'movie' if x == 'Movie' else 'tvSeries')
netflix['release_year'] = netflix['release_year'].astype(str)
netflix['title'] = netflix['title'].str.lower()

In [4]:
df = pd.merge(imdb, netflix, left_on=['primaryTitle', 'startYear'], right_on=['title', 'release_year'], how='inner')

In [5]:
print(netflix.shape)
print(df.shape)

(8807, 12)
(5551, 21)


In [56]:
import requests
from bs4 import BeautifulSoup
import json

header = {'User-Agent': 'Mozilla/5.0'}

s = requests.Session()
s.headers.update(header)

counter = 0

with open('data/movie_data.json', 'a') as f:
# For every title in the dataframe, send a request to IMDB using the tconst
    for index, row in df.iloc[:10, :].iterrows():
        try:
            response = s.get(f'https://www.imdb.com/title/{row["tconst"]}/')

            # Parse the response with BeautifulSoup
            soup = BeautifulSoup(response.text, 'html.parser')

            # Find the rating
            rating = soup.find('span', class_='sc-eb51e184-1 ljxVSS').text

            # Find the plot
            plot = soup.find('div', class_='ipc-html-content-inner-div').text

            # Find the review
            review = [r.text for r in soup.find('div', class_='ipc-html-content ipc-html-content--base').children][0]

            # Find the genres
            genres = [c.text for c in soup.find('div', class_='ipc-chip-list__scroller').children]

            # Define a dictionary to store the information
            movie_data = {
                'title': row['title'],
                'tconst': row['tconst'],
                'type': row['type'],
                'rating': rating,
                'plot': plot,
                'review': review,
                'genres': genres
            }

            # Save the data to a text file in JSON format
            f.write(json.dumps(movie_data, indent=4) + '\n')  # Append each movie as a new line of JSON

            counter += 1
            print(f"Processed {counter} of {df.shape[0]} movies.")
        except Exception as e:
            print(f"Error processing {row['title']}: {e}")

print("Movie data collection complete.")

In [53]:
with open('data/movie_data.json', 'r') as f:
    movies_data = json.load(f)

In [55]:
movies_data[0]

{'title': 'kate & leopold',
 'tconst': 'tt0035423',
 'rating': '9.0',
 'plot': 'Extreme parkour, spilled chili, and way too much baby talk! Head back to Dunder Mifflin and relive the funniest moments that happened before the opening credits of the classic sitcom.',
 'review': 'Honestly, I thought show as \'boring\' even before I watched a single episode. Seriously, what interesting could happen in an office environment? I remember first scene I saw from this show. It was from an internet wallpaper with Michael Scott with a white board behind him saying "You miss 100% of the shots you don\'t take".Then after years from that, I somehow started watching "The Office", I don\'t really remember why I did that. What I love most about this show is it\'s raw nature. The camera is kind of shaky yet still brings the best scenes...something that I had never seen before in a TV sitcom. It may seem less professional but adds real life, personal touch with the TV show and the characters in them.Obvio